# 数据的聚合和分组操作

## GroupBy机制

In [1]:
import numpy as np
import pandas as pd

df = pd.DataFrame({
    "key1":  ["a", "a", "b", "b", None],
    "key2": pd.Series([1, 2, 1, None, 3], dtype="Int64"),
    "data1": np.random.standard_normal(5),
    "data2": np.random.standard_normal(5),
})

df

,key1,key2,data1,data2
0,a,1,0.101987,1.963399
1,a,2,-0.310426,-0.724551
2,b,1,-0.714591,1.999235
3,b,<NA>,0.658837,-1.153476
4,None,3,1.459520,-0.234468


In [2]:
grouped = df["data1"].groupby(df["key1"])
grouped.mean()

key1
a   -0.104220
b   -0.027877
Name: data1, dtype: float64

In [3]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     1       0.101987
      2      -0.310426
b     1      -0.714591
Name: data1, dtype: float64

In [4]:
means.unstack()

key2,1,2
key1,,
a,0.101987,-0.310426
b,-0.714591,NaN


In [5]:
states = np.array(["OH", "CA", "CA", "OH", "TX"])
years = [2005, 2002, 2001, 2003, 2002]

df['data1'].groupby([states, years]).mean()

CA  2001   -0.714591
    2002   -0.310426
OH  2003    0.658837
    2005    0.101987
TX  2002    1.459520
Name: data1, dtype: float64

In [6]:
df.groupby('key1').mean()

,key2,data1,data2
key1,,,
a,1.5,-0.104220,0.619424
b,1.0,-0.027877,0.422880


In [7]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    1     0.101987  1.963399
     2    -0.310426 -0.724551
b    1    -0.714591  1.999235

In [8]:
df.groupby('key1').size()

key1
a    2
b    2
dtype: int64

In [9]:
df.groupby('key1', dropna=False).size()

key1
a      2
b      2
NaN    1
dtype: int64

In [10]:
df.groupby('key1').count()

,key2,data1,data2
key1,,,
a,2,2,2
b,1,2,2


In [11]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1  key2     data1     data2
0    a     1  0.101987  1.963399
1    a     2 -0.310426 -0.724551
b
  key1  key2     data1     data2
2    b     1 -0.714591  1.999235
3    b  <NA>  0.658837 -1.153476


In [12]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print(k1, k2)
    print(group)

a 1
  key1  key2     data1     data2
0    a     1  0.101987  1.963399
a 2
  key1  key2     data1     data2
1    a     2 -0.310426 -0.724551
b 1
  key1  key2     data1     data2
2    b     1 -0.714591  1.999235


In [13]:
pieces = {
    name: group for name, group in df.groupby('key1')
}
pieces['b']

,key1,key2,data1,data2
2,b,1,-0.714591,1.999235
3,b,<NA>,0.658837,-1.153476


In [14]:
for name, group in df.groupby('key1')['data1']:
    print(name)
    print(group)

a
0    0.101987
1   -0.310426
Name: data1, dtype: float64
b
2   -0.714591
3    0.658837
Name: data1, dtype: float64


In [15]:
for name, group in df.groupby('key1')[['data2']]:
    print(name)
    print(group)

a
      data2
0  1.963399
1 -0.724551
b
      data2
2  1.999235
3 -1.153476


In [16]:
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    1     1.963399
     2    -0.724551
b    1     1.999235

In [17]:
people = pd.DataFrame(np.random.standard_normal((5, 5)), columns=['a', 'b', 'c', 'd', 'e'], index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people

,a,b,c,d,e
Joe,1.920911,-0.146884,-1.279443,0.897184,0.021263
Steve,0.242243,-0.922172,-0.479605,-0.054210,0.607457
Wes,-0.098173,-1.531797,-0.275745,0.567333,0.917827
Jim,-0.397576,-0.916472,0.129651,-0.577445,0.004122
Travis,-1.512696,1.428854,-0.353971,-0.615234,0.064495


In [18]:
mapping = {
    'a': 'red',
    'b': 'blue',
    'c': 'blue',
    'd': 'red',
    'e': 'orange',
    'f': 'red',
}

by_columns = people.T.groupby(mapping)
by_columns.sum()

,Joe,Steve,Wes,Jim,Travis
blue,-1.426326,-1.401777,-1.807542,-0.786820,1.074883
orange,0.021263,0.607457,0.917827,0.004122,0.064495
red,2.818094,0.188034,0.469159,-0.975021,-2.127930


In [19]:
map_series = pd.Series(mapping)

people.T.groupby(map_series).count()

,Joe,Steve,Wes,Jim,Travis
blue,2,2,2,2,2
orange,1,1,1,1,1
red,2,2,2,2,2


In [20]:
people.groupby(len).count()

,a,b,c,d,e
3,3,3,3,3,3
5,1,1,1,1,1
6,1,1,1,1,1


In [21]:
key_list = ['one', 'one', 'two', 'two', 'one']
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one  1.920911 -0.146884 -1.279443  0.897184  0.021263
  two -0.397576 -1.531797 -0.275745 -0.577445  0.004122
5 one  0.242243 -0.922172 -0.479605 -0.054210  0.607457
6 one -1.512696  1.428854 -0.353971 -0.615234  0.064495

## 数据聚合

聚合指的是能够从数组产生标量值的任何数据转换过程

In [22]:
df

,key1,key2,data1,data2
0,a,1,0.101987,1.963399
1,a,2,-0.310426,-0.724551
2,b,1,-0.714591,1.999235
3,b,<NA>,0.658837,-1.153476
4,None,3,1.459520,-0.234468


In [23]:
grouped = df.groupby('key1')
grouped['data1'].nsmallest(1)

key1   
a     1   -0.310426
b     2   -0.714591
Name: data1, dtype: float64

In [24]:
def peak_to_peak(arr):
    return arr.max() - arr.min()
grouped.agg(peak_to_peak)

,key2,data1,data2
key1,,,
a,1,0.412413,2.687950
b,0,1.373428,3.152711


In [25]:
grouped.describe()

key2                                           data1            ...  \
     count mean       std  min   25%  50%   75%  max count      mean  ...   
key1                                                                  ...   
a      2.0  1.5  0.707107  1.0  1.25  1.5  1.75  2.0   2.0 -0.104220  ...   
b      1.0  1.0      <NA>  1.0   1.0  1.0   1.0  1.0   2.0 -0.027877  ...   

                         data2                                          \
           75%       max count      mean       std       min       25%   
key1                                                                     
a    -0.001116  0.101987   2.0  0.619424  1.900668 -0.724551 -0.052564   
b     0.315480  0.658837   2.0  0.422880  2.229304 -1.153476 -0.365298   

                                    
           50%       75%       max  
key1                                
a     0.619424  1.291411  1.963399  
b     0.422880  1.211058  1.999235  

[2 rows x 24 columns]

In [26]:
tips = pd.read_csv('examples/tips.csv')
tips.head()

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4


In [27]:
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [28]:
grouped = tips.groupby(['day', 'smoker'])
grouped_pct = grouped['tip_pct']
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [29]:
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [30]:
functions = ['count', 'mean', 'max']
results = grouped[['tip_pct', 'total_bill']].agg(functions)
results

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [31]:
grouped.agg({
    'tip': np.max,
    'size': 'count',
})

/var/folders/7v/zmfj4myd5gj8ttpbq6ynmdm80000gn/T/ipykernel_60365/1557363568.py:1: FutureWarning: The provided callable <function max at 0x106241630> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  grouped.agg({


tip  size
day  smoker             
Fri  No       3.50     4
     Yes      4.73    15
Sat  No       9.00    45
     Yes     10.00    42
Sun  No       6.00    57
     Yes      6.50    19
Thur No       6.70    45
     Yes      5.00    17

In [32]:
tips.groupby(['day', 'smoker'], as_index=False).mean(numeric_only=True).head()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113


In [33]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(column, ascending=False).head(n)
top(tips, n=6, column='total_bill')

,total_bill,tip,smoker,day,time,size,tip_pct
170,50.81,10.00,Yes,Sat,Dinner,3,0.196812
212,48.33,9.00,No,Sat,Dinner,4,0.186220
59,48.27,6.73,No,Sat,Dinner,4,0.139424
156,48.17,5.00,No,Sun,Dinner,6,0.103799
182,45.35,3.50,Yes,Sun,Dinner,3,0.077178
102,44.30,2.50,Yes,Sat,Dinner,3,0.056433


In [34]:
tips.groupby('smoker').apply(top)

/var/folders/7v/zmfj4myd5gj8ttpbq6ynmdm80000gn/T/ipykernel_60365/1695234324.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tips.groupby('smoker').apply(top)


total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     232       11.61  3.39     No   Sat  Dinner     2  0.291990
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       88        24.71  5.85     No  Thur   Lunch     2  0.236746
Yes    172        7.25  5.15    Yes   Sun  Dinner     2  0.710345
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       109       14.31  4.00    Yes   Sat  Dinner     2  0.279525

In [35]:
s = pd.Series(np.random.standard_normal(6))

s[::2] = np.nan
s

0         NaN
1    0.286297
2         NaN
3   -0.950976
4         NaN
5   -0.146547
dtype: float64

In [36]:
s.fillna(s.mean())

0   -0.270409
1    0.286297
2   -0.270409
3   -0.950976
4   -0.270409
5   -0.146547
dtype: float64

In [37]:
suits = ['红桃', '黑桃', '梅花', '方块']
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'Q']
cards = []
for suit in suits:
    cards.extend(suit + str(num) for num in base_names)
cards = pd.Series(card_val, index=cards)
cards.head()

红桃A    1
红桃2    2
红桃3    3
红桃4    4
红桃5    5
dtype: int64

In [38]:
def draw(deck, n = 5):
    return deck.sample(n)

draw(cards)

黑桃5    5
红桃5    5
梅花5    5
黑桃3    3
梅花8    8
dtype: int64

In [39]:
def get_suit(card):
    return card[-1]

cards.groupby(get_suit, group_keys=False).apply(draw, n=2)

黑桃10    10
红桃10    10
方块2      2
红桃2      2
方块3      3
红桃3      3
方块4      4
黑桃4      4
黑桃5      5
红桃5      5
红桃6      6
方块6      6
黑桃7      7
红桃7      7
方块8      8
红桃8      8
方块9      9
红桃9      9
红桃A      1
梅花A      1
梅花J     10
红桃J     10
红桃K     10
黑桃K     10
梅花Q     10
红桃Q     10
dtype: int64

In [40]:
df = pd.DataFrame({
    'category': ['a', 'a', 'a', 'b', 'b', 'b', 'c', 'c', 'c'],
    'data': np.random.standard_normal(9),
    'weights': np.random.randn(9),
})
df

,category,data,weights
0,a,-0.683462,-0.949823
1,a,-1.507668,1.371062
2,a,-1.007019,0.296991
3,b,-0.939374,1.852460
4,b,0.378432,-0.649496
5,b,0.776197,0.773223
6,c,-1.032327,1.745254
7,c,0.484534,-0.891310
8,c,-0.308327,-1.385583


In [41]:
grouped = df.groupby('category')

def get_wavg(group):
    return np.average(group['data'], weights=group['weights'])
grouped.apply(get_wavg)

/var/folders/7v/zmfj4myd5gj8ttpbq6ynmdm80000gn/T/ipykernel_60365/3461187993.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped.apply(get_wavg)


category
a   -2.390619
b   -0.701234
c    3.397664
dtype: float64

In [42]:
close_px = pd.read_csv('examples/stock_px.csv', parse_dates=True, index_col=0)
close_px.head()

,AAPL,MSFT,XOM,SPX
2003-01-02,7.40,21.11,29.22,909.03
2003-01-03,7.45,21.14,29.24,908.59
2003-01-06,7.45,21.52,29.96,929.01
2003-01-07,7.43,21.93,28.95,922.93
2003-01-08,7.28,21.31,28.83,909.93


In [43]:
rets = close_px.pct_change().dropna()
rets.head()

,AAPL,MSFT,XOM,SPX
2003-01-03,0.006757,0.001421,0.000684,-0.000484
2003-01-06,0.000000,0.017975,0.024624,0.022474
2003-01-07,-0.002685,0.019052,-0.033712,-0.006545
2003-01-08,-0.020188,-0.028272,-0.004145,-0.014086
2003-01-09,0.008242,0.029094,0.021159,0.019386


In [44]:
def get_year(x):
    return x.year

def spx_corr(group):
    return group.corrwith(group['SPX'])

by_year = rets.groupby(get_year)

by_year.apply(spx_corr)


,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [49]:
import statsmodels.api as sm

def regress(data, yvar=None, xvars=None):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params

by_year.apply(regress, yvar='AAPL', xvars=['SPX'])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


## 透视表和交叉表

In [50]:
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [ ]:
tips.pivot_table(index=['day', 'smoker'], values=['size', 'tip_pct', 'total_bill', 'tip'], aggfunc='mean')

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [66]:
tips.pivot_table(index=['time', 'day'], values=['tip_pct', 'size'], columns='smoker')

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

In [67]:
tips.pivot_table(index=['time', 'day'], columns='smoker', values=['tip_pct', 'size'], aggfunc='mean', margins=True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

In [68]:
tips.pivot_table(index=['time', 'smoker'], columns="day", values="tip_pct", aggfunc=len, margins=True)

day             Fri   Sat   Sun  Thur  All
time   smoker                             
Dinner No       3.0  45.0  57.0   1.0  106
       Yes      9.0  42.0  19.0   NaN   70
Lunch  No       1.0   NaN   NaN  44.0   45
       Yes      6.0   NaN   NaN  17.0   23
All            19.0  87.0  76.0  62.0  244

In [69]:
from io import StringIO

data = """Sample Nationality Handedness
1	USA    Right-handed
2	China  Left-handed
3	Japan  Right-handed
4	Germany    Right-handed
5	USA    Left-handed
6   China  Right-handed
7   Japan  Right-handed
8   China  Left-handed
9   USA    Right-handed"""

data = pd.read_table(StringIO(data), sep='\s+')
data.head()

,Sample,Nationality,Handedness
0,1,USA,Right-handed
1,2,China,Left-handed
2,3,Japan,Right-handed
3,4,Germany,Right-handed
4,5,USA,Left-handed


In [70]:
pd.crosstab(data['Nationality'], data['Handedness'], margins=True)

Handedness,Left-handed,Right-handed,All
Nationality,,,
China,2,1,3
Germany,0,1,1
Japan,0,2,2
USA,1,2,3
All,3,6,9


In [71]:
pd.crosstab([tips['time'], tips['day']], tips['smoker'], margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244